In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l')
data.generate_data('letters_word_accetuation_train',
                   'letters_word_accetuation_test',
                   'letters_word_accetuation_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [21]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 7
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
amejnavejob
Ncndi-
Ncndi-
[3]


In [7]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 60
num_fake_epoch = 20




conv_input_shape=(23, 36)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(43, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [8]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/1200
1345/1344 [==============================] - 36s - loss: 0.2069 - actual_accuracy: 0.4066 - val_loss: 0.1468 - val_actual_accuracy: 0.6223
Epoch 2/1200
1345/1344 [==============================] - 35s - loss: 0.1482 - actual_accuracy: 0.5730 - val_loss: 0.1251 - val_actual_accuracy: 0.6660
Epoch 3/1200
1345/1344 [==============================] - 39s - loss: 0.1328 - actual_accuracy: 0.6419 - val_loss: 0.1161 - val_actual_accuracy: 0.7013
Epoch 4/1200
1345/1344 [==============================] - 42s - loss: 0.1235 - actual_accuracy: 0.6783 - val_loss: 0.1044 - val_actual_accuracy: 0.7469
Epoch 5/1200
1345/1344 [==============================] - 37s - loss: 0.1162 - actual_accuracy: 0.6983 - val_loss: 0.1045 - val_actual_accuracy: 0.7623
Epoch 6/1200
1345/1344 [==============================] - 35s - loss: 0.1107 - actual_accuracy: 0.7182 - val_loss: 0.0940 - val_actual_accuracy: 0.7845
Epoch 7/1200
1345/1344 [==============================] - 34s - loss: 0.1058 - actual_ac

1345/1344 [==============================] - 44s - loss: 0.0313 - actual_accuracy: 0.9257 - val_loss: 0.0702 - val_actual_accuracy: 0.8903
Epoch 55/1200
1345/1344 [==============================] - 40s - loss: 0.0308 - actual_accuracy: 0.9284 - val_loss: 0.0742 - val_actual_accuracy: 0.8938
Epoch 56/1200
1345/1344 [==============================] - 38s - loss: 0.0299 - actual_accuracy: 0.9306 - val_loss: 0.0770 - val_actual_accuracy: 0.8804
Epoch 57/1200
1345/1344 [==============================] - 45s - loss: 0.0282 - actual_accuracy: 0.9345 - val_loss: 0.0764 - val_actual_accuracy: 0.8777
Epoch 58/1200
1345/1344 [==============================] - 40s - loss: 0.0287 - actual_accuracy: 0.9335 - val_loss: 0.0728 - val_actual_accuracy: 0.8903
Epoch 59/1200
1345/1344 [==============================] - 44s - loss: 0.0276 - actual_accuracy: 0.9342 - val_loss: 0.0779 - val_actual_accuracy: 0.8828
Epoch 60/1200
1345/1344 [==============================] - 37s - loss: 0.0283 - actual_accuracy:

1345/1344 [==============================] - 39s - loss: 0.0163 - actual_accuracy: 0.9638 - val_loss: 0.1069 - val_actual_accuracy: 0.8804
Epoch 108/1200
1345/1344 [==============================] - 44s - loss: 0.0164 - actual_accuracy: 0.9634 - val_loss: 0.0952 - val_actual_accuracy: 0.8857
Epoch 109/1200
1345/1344 [==============================] - 40s - loss: 0.0162 - actual_accuracy: 0.9642 - val_loss: 0.0831 - val_actual_accuracy: 0.8984
Epoch 110/1200
1345/1344 [==============================] - 42s - loss: 0.0160 - actual_accuracy: 0.9649 - val_loss: 0.1037 - val_actual_accuracy: 0.8953
Epoch 111/1200
1345/1344 [==============================] - 42s - loss: 0.0158 - actual_accuracy: 0.9645 - val_loss: 0.0932 - val_actual_accuracy: 0.8915
Epoch 112/1200
1345/1344 [==============================] - 41s - loss: 0.0157 - actual_accuracy: 0.9646 - val_loss: 0.1003 - val_actual_accuracy: 0.8861
Epoch 113/1200
1345/1344 [==============================] - 43s - loss: 0.0155 - actual_acc

1345/1344 [==============================] - 49s - loss: 0.0120 - actual_accuracy: 0.9731 - val_loss: 0.0937 - val_actual_accuracy: 0.8953
Epoch 161/1200
1345/1344 [==============================] - 40s - loss: 0.0117 - actual_accuracy: 0.9741 - val_loss: 0.0830 - val_actual_accuracy: 0.9099
Epoch 162/1200
1345/1344 [==============================] - 44s - loss: 0.0112 - actual_accuracy: 0.9758 - val_loss: 0.0933 - val_actual_accuracy: 0.9049
Epoch 163/1200
1345/1344 [==============================] - 48s - loss: 0.0122 - actual_accuracy: 0.9741 - val_loss: 0.0993 - val_actual_accuracy: 0.8934
Epoch 164/1200
1345/1344 [==============================] - 40s - loss: 0.0113 - actual_accuracy: 0.9756 - val_loss: 0.1038 - val_actual_accuracy: 0.8946
Epoch 165/1200
1345/1344 [==============================] - 49s - loss: 0.0114 - actual_accuracy: 0.9739 - val_loss: 0.1258 - val_actual_accuracy: 0.8765
Epoch 166/1200
1345/1344 [==============================] - 44s - loss: 0.0107 - actual_acc

1345/1344 [==============================] - 44s - loss: 0.0083 - actual_accuracy: 0.9817 - val_loss: 0.1071 - val_actual_accuracy: 0.9051
Epoch 214/1200
1345/1344 [==============================] - 48s - loss: 0.0092 - actual_accuracy: 0.9800 - val_loss: 0.1147 - val_actual_accuracy: 0.8900
Epoch 215/1200
1345/1344 [==============================] - 46s - loss: 0.0091 - actual_accuracy: 0.9800 - val_loss: 0.1051 - val_actual_accuracy: 0.9038
Epoch 216/1200
1345/1344 [==============================] - 43s - loss: 0.0085 - actual_accuracy: 0.9807 - val_loss: 0.1027 - val_actual_accuracy: 0.8984
Epoch 217/1200
1345/1344 [==============================] - 48s - loss: 0.0082 - actual_accuracy: 0.9824 - val_loss: 0.1006 - val_actual_accuracy: 0.9107
Epoch 218/1200
1345/1344 [==============================] - 44s - loss: 0.0090 - actual_accuracy: 0.9810 - val_loss: 0.0923 - val_actual_accuracy: 0.9072
Epoch 219/1200
1345/1344 [==============================] - 44s - loss: 0.0086 - actual_acc

...

1345/1344 [==============================] - 46s - loss: 0.0041 - actual_accuracy: 0.9923 - val_loss: 0.0995 - val_actual_accuracy: 0.9214
Epoch 956/1200
1345/1344 [==============================] - 48s - loss: 0.0036 - actual_accuracy: 0.9929 - val_loss: 0.1240 - val_actual_accuracy: 0.9222
Epoch 957/1200
1345/1344 [==============================] - 44s - loss: 0.0037 - actual_accuracy: 0.9918 - val_loss: 0.1208 - val_actual_accuracy: 0.9049
Epoch 958/1200
1345/1344 [==============================] - 47s - loss: 0.0033 - actual_accuracy: 0.9937 - val_loss: 0.1153 - val_actual_accuracy: 0.9164
Epoch 959/1200
1345/1344 [==============================] - 48s - loss: 0.0033 - actual_accuracy: 0.9930 - val_loss: 0.1353 - val_actual_accuracy: 0.9103
Epoch 960/1200
1345/1344 [==============================] - 46s - loss: 0.0036 - actual_accuracy: 0.9923 - val_loss: 0.1182 - val_actual_accuracy: 0.9099
Epoch 961/1200
1345/1344 [==============================] - 42s - loss: 0.0034 - actual_acc

1345/1344 [==============================] - 48s - loss: 0.0032 - actual_accuracy: 0.9937 - val_loss: 0.1232 - val_actual_accuracy: 0.9081
Epoch 1009/1200
1345/1344 [==============================] - 46s - loss: 0.0031 - actual_accuracy: 0.9937 - val_loss: 0.1297 - val_actual_accuracy: 0.9026
Epoch 1010/1200
1345/1344 [==============================] - 46s - loss: 0.0037 - actual_accuracy: 0.9926 - val_loss: 0.1061 - val_actual_accuracy: 0.9114
Epoch 1011/1200
1345/1344 [==============================] - 46s - loss: 0.0030 - actual_accuracy: 0.9937 - val_loss: 0.1186 - val_actual_accuracy: 0.9053
Epoch 1012/1200
1345/1344 [==============================] - 45s - loss: 0.0032 - actual_accuracy: 0.9932 - val_loss: 0.1552 - val_actual_accuracy: 0.8880
Epoch 1013/1200
1345/1344 [==============================] - 42s - loss: 0.0034 - actual_accuracy: 0.9929 - val_loss: 0.1168 - val_actual_accuracy: 0.9206
Epoch 1014/1200
1345/1344 [==============================] - 46s - loss: 0.0038 - actu

1345/1344 [==============================] - 50s - loss: 0.0036 - actual_accuracy: 0.9928 - val_loss: 0.1268 - val_actual_accuracy: 0.9145
Epoch 1061/1200
1345/1344 [==============================] - 44s - loss: 0.0035 - actual_accuracy: 0.9930 - val_loss: 0.1417 - val_actual_accuracy: 0.9011
Epoch 1062/1200
1345/1344 [==============================] - 48s - loss: 0.0034 - actual_accuracy: 0.9932 - val_loss: 0.1322 - val_actual_accuracy: 0.9122
Epoch 1063/1200
1345/1344 [==============================] - 49s - loss: 0.0040 - actual_accuracy: 0.9926 - val_loss: 0.1347 - val_actual_accuracy: 0.9114
Epoch 1064/1200
1345/1344 [==============================] - 48s - loss: 0.0031 - actual_accuracy: 0.9938 - val_loss: 0.1319 - val_actual_accuracy: 0.9064
Epoch 1065/1200
1345/1344 [==============================] - 50s - loss: 0.0029 - actual_accuracy: 0.9937 - val_loss: 0.1202 - val_actual_accuracy: 0.9107
Epoch 1066/1200
1345/1344 [==============================] - 49s - loss: 0.0036 - actu

1345/1344 [==============================] - 50s - loss: 0.0031 - actual_accuracy: 0.9941 - val_loss: 0.1148 - val_actual_accuracy: 0.9126
Epoch 1113/1200
1345/1344 [==============================] - 47s - loss: 0.0039 - actual_accuracy: 0.9929 - val_loss: 0.1086 - val_actual_accuracy: 0.9145
Epoch 1114/1200
1345/1344 [==============================] - 49s - loss: 0.0032 - actual_accuracy: 0.9929 - val_loss: 0.1317 - val_actual_accuracy: 0.9091
Epoch 1115/1200
1345/1344 [==============================] - 50s - loss: 0.0033 - actual_accuracy: 0.9932 - val_loss: 0.1318 - val_actual_accuracy: 0.9091
Epoch 1116/1200
1345/1344 [==============================] - 49s - loss: 0.0030 - actual_accuracy: 0.9937 - val_loss: 0.1243 - val_actual_accuracy: 0.9099
Epoch 1117/1200
1345/1344 [==============================] - 46s - loss: 0.0038 - actual_accuracy: 0.9924 - val_loss: 0.1022 - val_actual_accuracy: 0.9199
Epoch 1118/1200
1345/1344 [==============================] - 49s - loss: 0.0030 - actu

1345/1344 [==============================] - 50s - loss: 0.0029 - actual_accuracy: 0.9946 - val_loss: 0.1472 - val_actual_accuracy: 0.9057
Epoch 1165/1200
1345/1344 [==============================] - 49s - loss: 0.0035 - actual_accuracy: 0.9928 - val_loss: 0.1350 - val_actual_accuracy: 0.9141
Epoch 1166/1200
1345/1344 [==============================] - 49s - loss: 0.0033 - actual_accuracy: 0.9934 - val_loss: 0.1139 - val_actual_accuracy: 0.9210
Epoch 1167/1200
1345/1344 [==============================] - 44s - loss: 0.0026 - actual_accuracy: 0.9943 - val_loss: 0.1369 - val_actual_accuracy: 0.9095
Epoch 1168/1200
1345/1344 [==============================] - 45s - loss: 0.0032 - actual_accuracy: 0.9936 - val_loss: 0.1213 - val_actual_accuracy: 0.9130
Epoch 1169/1200
1345/1344 [==============================] - 45s - loss: 0.0039 - actual_accuracy: 0.9926 - val_loss: 0.1079 - val_actual_accuracy: 0.9153
Epoch 1170/1200
1345/1344 [==============================] - 45s - loss: 0.0035 - actu

In [6]:
name = '60_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [6]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()